In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
import time

In [6]:
k = 7
m = 4 

In [4]:
def create_matrix_A(f, n, num_type=np.float64):
    return np.array([[f(i+1, j+1) for j in range(n)] for i in range(n)], dtype=num_type)

def create_vector_b(n, num_type=np.float64):
    return np.random.choice([-1, 1], size=n)

def f(i, j):
    if i == j:
        return k
    else:
        return 1/(abs(i-j)+m)

In [19]:
n = 5
A = create_matrix_A(f, n)
x = create_vector_b(n)
b = A @ x
b

array([ 7.38452381,  7.42380952,  7.4       ,  7.30952381, -6.36547619])

In [63]:
def progress_diff(current, next, rho, A=None, b=None):
    return np.linalg.norm(next - current) < rho

def sol_diff(current, A, b, rho, next=None):
    return np.linalg.norm(A @ current - b) < rho

def jacob_method(A, b, start, stop=progress_diff, rho=1e-6, max_iterations=1e3):
    d = np.diag(A)
    r = A - np.diagflat(d)
    current = start
    iterations = 0
    while iterations < max_iterations:
        next = (b - (r @ current)) / d
        if stop(current=current, next=next, rho=rho, A=A, b=b): 
            break
        current = next
        iterations += 1
    return next, iterations
    


In [64]:
start = np.array([0]*5)
x, it = jacob_method(A, b, start, sol_diff)
x, it

(array([ 0.99999999,  0.99999999,  0.99999999,  0.99999999, -1.00000001]), 7)

In [85]:
def error_calc(x1, x2):
    return np.linalg.norm(x1 - x2)

def solution(n, rho, start_vec, stop=progress_diff, max_iterations=1e3):
    results = {}

    vectors = [[None]*len(rho) for _ in range(len(n))]
    errors = np.empty((len(n), len(rho)), np.float64)
    iterations = np.empty((len(n), len(rho)), int)
    times = np.empty((len(n), len(rho)), np.float64)

    for i, ni in enumerate(n):
        A = create_matrix_A(f, ni)
        x_start = start_vec(ni)
        x_sol = create_vector_b(ni)
        b = A @ x_sol
        for j, rhoj in enumerate(rho):
            st_time = time.time()
            x, it = jacob_method(A, b, x_start, stop=stop, rho=rhoj, max_iterations=max_iterations)
            end_time = time.time()
            if it >= 0:
                vectors[i][j] = x
                errors[i, j] = error_calc(x, x_sol)
                iterations[i, j] = it
                times[i, j] = end_time - st_time
            else:
                vectors[i][j] = None
                errors[i, j] = None
                iterations[i, j] = None
                times[i, j] = None
    
    for key, value in zip(['vector', 'error', 'iter', 'time'], [vectors, errors, iterations, times]):
        results[key] = pd.DataFrame(value, index=n, columns=rho)

    return results


In [106]:
n = list(range(2, 500))
rho = [1e-8, 1e-10, 1e-12, 1e-14, 1e-16]
stop = progress_diff
start_vec = lambda x: np.array([0]*x)
res1 = solution(n, rho, start_vec, stop=stop)

In [112]:
key = 'time'
pr1 = res1[key].iloc[:10]
pl1 = res1[key].iloc[10::50]
r1 = pd.concat([pr1, pl1])
r1.to_csv('time_prog.csv')

In [89]:
res2 = solution(n, rho, start_vec, stop=sol_diff)

In [118]:
key = 'time'
pr2 = res2[key].iloc[:10]
pl2 = res2[key].iloc[10::50]
r2 = pd.concat([pr2, pl2])
r2.to_csv('time_sol.csv')

In [94]:
n3 = list(range(2,100))
start_vec3 = lambda x: np.random.random(x)
res3 = solution(n3, rho, start_vec3, stop=sol_diff)

In [120]:
key = 'error'
pr3 = res3[key].iloc[:10]
pl3 = res3[key].iloc[10::50]
r3 = pd.concat([pr3, pl3])
r3.to_csv('error_sol_rv.csv')

In [95]:
res4 = solution(n3, rho, start_vec3, stop=progress_diff)

In [121]:
key = 'error'
pr4 = res4[key].iloc[:10]
pl4 = res4[key].iloc[10::50]
r4 = pd.concat([pr4, pl4])
r4.to_csv('error_prog_rv.csv')

In [1]:
def spectral_radius(n):
    A = create_matrix_A(f, n)
    b = np.diagflat(np.diag(A))
    M = np.eye(n) - np.linalg.inv(b) @ A
    return np.max(np.abs(np.linalg.eigvals(A)))

In [7]:
specs = np.array([spectral_radius(i) for i in range(2, 500)])

In [8]:
specs

array([ 7.2       ,  7.37819677,  7.53920673,  7.68628567,  7.82181246,
        7.94758309,  8.06499054,  8.17513939,  8.27892196,  8.37707052,
        8.47019413,  8.55880537,  8.64334006,  8.72417215,  8.80162508,
        8.87598066,  8.94748604,  9.01635924,  9.08279366,  9.14696167,
        9.20901763,  9.26910031,  9.327335  ,  9.3838352 ,  9.43870406,
        9.49203565,  9.54391597,  9.59442389,  9.64363191,  9.69160686,
        9.73841045,  9.78409979,  9.82872787,  9.87234393,  9.9149938 ,
        9.95672024,  9.99756317, 10.03755997, 10.07674565, 10.11515307,
       10.1528131 , 10.18975479, 10.22600551, 10.26159108, 10.29653588,
       10.33086295, 10.36459411, 10.39775002, 10.43035028, 10.46241349,
       10.49395732, 10.52499858, 10.55555326, 10.5856366 , 10.61526313,
       10.64444669, 10.67320052, 10.70153727, 10.72946901, 10.75700731,
       10.78416323, 10.8109474 , 10.83736996, 10.86344068, 10.88916891,
       10.91456364, 10.93963351, 10.96438681, 10.98883154, 11.01

In [124]:
df_spec = pd.DataFrame(specs, index=list(range(2, 500)), columns=["promien spektralny"])
df_spec1 = df_spec.iloc[:10]
df_spec2 = df_spec.iloc[10::50]
df_spec = pd.concat([df_spec1, df_spec2])
df_spec.to_csv("spec_rad.csv")